In [13]:
# This script is used for testing all adaptations and changes made

In [1]:
import pandas as pd
import numpy as np

from scipy import linalg
from scipy import stats

In [2]:
# Load data for change detection as a pandas data frame.
infys_df =  pd.read_csv("../data/iMAD/iMAD_Data_12_Constant_Zeros.csv") #need to change the dataset 
infys_df.head()

,Unnamed: 0,File,Cluster_ID,AvgDbh,AvgCrownDiameter,AvgCrownHeight,AvgCrownArea,SpeciesCount,AvgTreeHeight,TreeCount,J,X,Y
0,1,1,77652,16.103571,3.937736,3.367925,15.409993,8,8.555357,56,0.430012,-92.433389,15.588935
1,2,1,28451,10.380952,2.209524,0.000000,3.956920,2,2.909524,21,0.276195,-99.546778,26.072916
2,3,1,51197,12.280889,3.199010,5.360500,9.575256,32,9.168889,225,0.769687,-88.688611,20.654722
3,4,1,53152,12.208163,3.172527,3.315385,8.666242,7,5.419388,98,0.649813,-102.826388,20.825555
4,5,1,55242,14.996429,4.439683,4.557937,17.199388,5,5.458333,84,0.529170,-104.218713,20.403546


In [3]:
# Create new ids by concatenating 'Conglomerado' and 'Sitio'.
    ## don't think I need to do that
    ## Use existing ID of Metadataset called Cluster_ID -> should be able to leave that one out


# As we want to compare cycle 1 vs cycle 2 lets separate them.   
## Stays the same for long dataformat
## might figure out how to 
infys_df_c1 = infys_df[infys_df['File'] == 1]
infys_df_c2 = infys_df[infys_df['File'] == 2]

In [4]:
# Find intersection of new ID variable: 'cng_sit', so obs. that are present in both cycles.
s1 = infys_df_c1['Cluster_ID']
s2 = infys_df_c2['Cluster_ID']
cluster_id_inter = pd.Series(list(set(s1).intersection(set(s2))))
## might have already done this in R by using my filter(muestreado1 == 1 & muestreado2 == 1)



In [5]:
# Filter out obs. that dont appear in cng_sit_inter.
infys_df_c1 = infys_df_c1[infys_df_c1['Cluster_ID'].isin(cluster_id_inter)]
infys_df_c2 = infys_df_c2[infys_df_c2['Cluster_ID'].isin(cluster_id_inter)]

In [6]:
# Select only variables to be utilized in change detection.
## probably lose H and add tree counts and species counts
vars = ["SpeciesCount", "TreeCount", "J", "AvgTreeHeight", "AvgDbh", "AvgCrownDiameter", "AvgCrownHeight", "AvgCrownArea"]
nvars = len(vars)
infys_df_c1_c = infys_df_c1[infys_df_c1.columns.intersection(vars)]
infys_df_c2_c = infys_df_c2[infys_df_c2.columns.intersection(vars)]

c1_c_nans = infys_df_c1_c.isna().any(axis=1).to_numpy()
c2_c_nans = infys_df_c2_c.isna().any(axis=1).to_numpy()

In [7]:
# Create change data matrix, missing values mask and initial change weights.
## think this can stay the same
dm = np.zeros((2 * nvars, infys_df_c1_c.shape[0]))
dm[0:nvars] = np.transpose(infys_df_c1_c.to_numpy())
dm[nvars:] = np.transpose(infys_df_c2_c.to_numpy())

nodataidx = c1_c_nans | c2_c_nans
gooddataidx = nodataidx == False
dm = dm[:, gooddataidx]
ngood = np.sum(gooddataidx)

In [17]:
# Change detection iterations.
## also think this can stay -> can't: have to manage NAs for empty plots without throwing them out completely
# iteration of MAD   
wt = np.ones(int(ngood))
delta = 1.0
oldrho = np.zeros(nvars)
iter = 0
 
while (delta > 0.01) and (iter < 35): 
    print(iter)
    # Weighted covariance matrices and means.
    sumw = np.sum(wt)
    means = np.average(dm,axis=1, weights=wt)
    dmc = dm - means[:,np.newaxis]
    dmc = np.multiply(dmc,np.sqrt(wt))
    sigma = np.dot(dmc,dmc.T)/sumw
   
    s11 = sigma[0:nvars, 0:nvars]
    s22 = sigma[nvars:, nvars:]
    s12 = sigma[0:nvars, nvars:]
    s21 = sigma[nvars:, 0:nvars]
    
    # Solution of generalized eigenproblems.
    aux_1 = linalg.solve(s22, s21)
    lama, a = linalg.eig(np.dot(s12, aux_1), s11)
    aux_2 = linalg.solve(s11, s12)
    lamb, b = linalg.eig(np.dot(s21, aux_2), s22)
    
    
    # Sort a.  
    idx = np.argsort(lama)
    a = a[:, idx]
    
    # Sort b.        
    idx = np.argsort(lamb)
    b = b[:, idx]
    
    # Canonical correlations.        
    rho = np.sqrt(np.real(lamb[idx]))
    
    
    # Normalize dispersions.  
    tmp1 = np.dot(np.dot(a.T,s11), a)
    tmp2 = 1. / np.sqrt(np.diag(tmp1))
    tmp3 = np.tile(tmp2, (nvars, 1))
    a = np.multiply(a, tmp3)
    b = np.mat(b)
    tmp1 = np.dot(np.dot(b.T,s22), b)
    tmp2 = 1. / np.sqrt(np.diag(tmp1))
    tmp3 = np.tile(tmp2, (nvars, 1))
    b = np.multiply(b, tmp3)
        
    # Assure positive correlation
    tmp = np.diag(np.dot(np.dot(a.T,s12), b))
    b = np.dot(b,np.diag(tmp / np.abs(tmp)))

    # Canonical and MAD variates
    U = np.dot(a.T , (dm[0:nvars, :] - means[0:nvars, np.newaxis]))    
    V = np.dot(b.T , (dm[nvars:, :] - means[nvars:, np.newaxis]))          
    MAD = U - V  
    
            
    # New weights.        
    var_mad = np.tile(np.mat(2 * (1 - rho)).T, (1, ngood))    
    chisqr = np.sum(np.multiply(MAD, MAD) / var_mad, 0)
    wt = np.squeeze(1 - np.array(stats.chi2._cdf(chisqr, nvars)))
    
        
    # Continue iteration.        
    delta = np.sum(np.abs(rho - oldrho))
    oldrho = rho
    iter += 1
    
# reshape to original image size, by including nodata pixels    
MADout = np.zeros((int(nvars), infys_df_c1_c.shape[0]))
MADout[0:nvars, gooddataidx] = MAD
MADout[nvars:(nvars + 1), gooddataidx] = chisqr

[0.00369928 0.23580311 0.36691833 0.48350385 0.63753748 0.73831177
 0.77522007 0.88424134]
[0.12195559 0.45322532 0.65234642 0.69315469 0.82537807 0.85992517
 0.89086983 0.9519858 ]
[0.35587625 0.55170227 0.78828637 0.79880733 0.88921561 0.91035235
 0.93576451 0.97291047]
[0.46692447 0.62843701 0.8379767  0.85703376 0.91887746 0.93612445
 0.95660532 0.98248155]
[0.5271417  0.692524   0.86834484 0.88585192 0.93591707 0.9501768
 0.96803127 0.98753647]
[0.56323456 0.73784036 0.88950329 0.90461736 0.94693302 0.95917598
 0.97480667 0.99044574]
[0.59087233 0.76943094 0.90454048 0.91831428 0.95437632 0.96559981
 0.97911897 0.9921896 ]
[0.61366003 0.79419033 0.9155306  0.92871448 0.95974126 0.97039557
 0.98202308 0.9933004 ]
[0.63297469 0.81460556 0.92379045 0.93666301 0.96386541 0.97400646
 0.98409085 0.99406603]
[0.64982391 0.83187431 0.93017418 0.94277714 0.96719899 0.97672563
 0.98564524 0.9946286 ]
[0.66504913 0.84711042 0.93527669 0.94760618 0.96999593 0.97881693
 0.98687297 0.99506337]


In [9]:
# TODO add coordinates to table
# Pandas copied R maybe read a bit on python data frame manipulation
## should also already work - especially with the single source coordinates from the metadata
MADout = np.transpose(MADout).copy()
MADout_df = pd.DataFrame(MADout, columns=vars)

# Add chi squared variable to table
chisqr = np.transpose(chisqr).copy()
Test_df = pd.DataFrame(chisqr)
MADout_df["chisqr"] = Test_df[0]

MADout_df.head()


,SpeciesCount,TreeCount,J,AvgTreeHeight,AvgDbh,AvgCrownDiameter,AvgCrownHeight,AvgCrownArea,chisqr
0,0.479148,-1.442019,2.746079,0.466858,-0.062241,1.540801,1.272953,0.041310,-1.287494e+15
1,0.476009,0.162200,-0.068169,0.114847,0.902737,-0.119318,0.507231,-1.332967,5.104861e+14
2,-0.218807,1.340997,-0.476434,0.109045,-0.360241,-0.665795,-0.588026,0.567759,-1.818717e+14
3,0.520303,-0.216383,0.531815,0.675355,0.730902,0.409999,-0.505379,-0.507542,5.714046e+14
4,-2.532363,0.124608,-1.495797,-0.343277,0.164959,-0.025257,-2.135551,-0.329093,-1.997008e+14


In [10]:
MADout_df["X"] = infys_df_c1["X"]
MADout_df["Y"] = infys_df_c1["Y"]

In [11]:
MADout_df.to_csv('iMAD_results_12_Constant_Zeros.csv', index=False)

In [54]:
## Beta

MADout_df["Y"] = infys_df_c1["std_chg"]
np.std(madout(lstone))
madout(lastone <2*np.std)=2

SyntaxError: cannot assign to function call here. Maybe you meant '==' instead of '='? (54363561.py, line 5)